In [1]:
#思路 按染色体对 整个数据集求和得伪bulk矩阵，借用dchic的pc值校正方法，定义单独的函数，输入为多个pc，输入为校正后的最终Pc值，所有的cell和bulk数据共用相同的投影矩阵
import subprocess
import sys
sys.path.append('/home/higashi/Higashi/higashi')


from Higashi_backend.utils import *
from Higashi_analysis.Higashi_analysis import *
import h5py
from sklearn.preprocessing import MinMaxScaler, quantile_transform
import os

os.environ["OMP_NUM_THREADS"] = "10"
import pandas as pd
import argparse

def create_mask(k=30, chrom="chr1", origin_sparse=None):
	final = np.array(np.sum(origin_sparse, axis=0).todense())
	size = origin_sparse[0].shape[-1]
	#生成一个稀疏矩阵尺寸的全为0的矩阵
	a = np.zeros((size, size))
	#默认调用时传递k = 100000
	#如果该染色体被划分为小于10万个bin 则相当于还是全0矩阵
	#
	if k > 0:
		for i in range(min(k, len(a))):
			for j in range(len(a) - i):
				a[j, j + i] = 1
				a[j + i, j] = 1
		a = np.ones_like((a)) - a

	# print(a)
	# 对bulk矩阵每一行求和，如果对应行的和等于0 则为true 否则为false
	gap = np.sum(final, axis=-1, keepdims=False) == 0

	# 如果cytoband文件存在
	if cytoband_path is not None:
		#读取该文件为表格
		gap_tab = pd.read_table(cytoband_path, sep="\t", header=None)
		#定义列名
		gap_tab.columns = ['chrom', 'start', 'end', 'name', 'type']
		#获取name列
		name = np.array(gap_tab['name'])
		# print (name)
		#qC1.1
		#获取name 第一个字符
		pqarm = np.array([str(s)[0] for s in name])
		#作为gap_tab的新列
		gap_tab['pq_arm'] = pqarm
		#获取 区域长度作为新列
		gap_tab['length'] = gap_tab['end'] - gap_tab['start']
		#按照 'chrom' 和 'pq_arm' 分组并对数值列进行求和
		summarize = gap_tab.groupby(['chrom', 'pq_arm']).sum().reset_index()
		# print (summarize)
		#如果求和结果中['pq_arm'] == 'p'的和大于0
		#则划分点为该染色体中 对符合条件（summarize['pq_arm'] == 'p')）的 'length' 列数据每个元素除以 分辨率 并向上取整后的第一个元素
		if np.sum(summarize['pq_arm'] == 'p') > 0:
			split_point = \
			np.ceil(np.array(summarize[(summarize['chrom'] == chrom) & (summarize['pq_arm'] == 'p')]['length']) / res)[0]
		#否则 置划分点为-1
		else:
			split_point = -1
		
		gap_list = gap_tab[(gap_tab["chrom"] == chrom) & (gap_tab["type"] == "acen")]
		start = np.floor((np.array(gap_list['start'])) / res).astype('int')
		end = np.ceil((np.array(gap_list['end'])) / res).astype('int')
		
		for s, e in zip(start, end):
			a[s:e, :] = 1
			a[:, s:e] = 1
	#如果cytoband文件不存在
	#直接置split_point = -1
	else:
		split_point = -1
	#将矩阵a 中gap为true的对应位置置1，如：gap第一个位置为true。则a的第一行 第一列置为1
	a[gap, :] = 1
	a[:, gap] = 1
	#返回结果a 和转换为整数的划分点
	return a, int(split_point)


def test_compartment(matrix, return_PCA=False, model=None, expected = None):
	contact = matrix
	# np.fill_diagonal(contact, np.max(contact))
	# contact = KRnormalize(matrix)
	# contact[np.isnan(contact)] = 0.0
	contact = sqrt_norm(matrix)
	contact = oe(contact, expected)
	np.fill_diagonal(contact, 1)
	with warnings.catch_warnings():
		warnings.filterwarnings(
			"ignore", category=PearsonRConstantInputWarning
		)
		contact = pearson(contact)
	np.fill_diagonal(contact, 1)
	contact[np.isnan(contact)] = 0.0
	if model is not None:
		y = model.transform(contact)
	else:
		# pca = PCA(n_components=3)       #保留的pca的列数
		pca = PCA(n_components=2)
		y = pca.fit_transform(contact)
	if return_PCA:
		return y, pca
	else:
		return y

def get_config(config_path = "./config.jSON"):
	c = open(config_path,"r")
	return json.load(c)

# res = 250000
# temp_dir = '/home/dataset/snm3c/250k/raw'
# output = 'scCompartment_snsm3c250k'
# cytoband_path = '/home/annotation/hg19/cytoBand.txt'

# chrom_list = ['chr1']
# chrom_list = np.array(chrom_list)

def download_file_with_curl(url, destination):
    subprocess.run(['curl', '-O', url, '-L'], check=True, cwd=destination)


def download_and_extract(genome, folder, resolution):
    if folder is None or folder.strip() == "":
        folder = f"{genome}_{int(resolution)}_goldenpathData"
        if not os.path.exists(folder):
            os.makedirs(folder)
            current_path = os.getcwd()
            os.chdir(current_path)
        folder = os.path.normpath(folder)
    else:
        folder = os.path.normpath(folder)

    genome_fa_file = f"{folder}/{genome}.fa"
    if not os.path.exists(f"{folder}/cytoBand.txt.gz"):
        print("download cytoBand.txt.gz")
        download_file_with_curl(f"http://hgdownload.cse.ucsc.edu/goldenPath/{genome}/database/cytoBand.txt.gz", folder)
        

    if not os.path.exists(f"{folder}/{genome}.chrom.sizes"):
        print(f"download {genome}.chrom.sizes")
        download_file_with_curl(f"http://hgdownload.cse.ucsc.edu/goldenPath/{genome}/bigZips/{genome}.chrom.sizes", folder)
    if not os.path.exists(f"{folder}/{genome}.refGene.gtf.gz"):
        print(f"download {genome}.refGene.gtf.gz")
        download_file_with_curl(f"http://hgdownload.cse.ucsc.edu/goldenPath/{genome}/bigZips/genes/{genome}.refGene.gtf.gz", folder)
        

    if not os.path.exists(f"{folder}/{genome}.fa.gz"):
        genome_fa_url = f"http://hgdownload.cse.ucsc.edu/goldenPath/{genome}/bigZips/{genome}.fa.gz"
        print(f"download {genome}.fa.gz")
        download_file_with_curl(genome_fa_url, folder)
    if not os.path.exists(f"{folder}/{genome}.fa"):  
        print(f"Unzipping {genome}.fa")
        #解压文件输出定向到{folder}文件夹下 {genome}.fa文件
        subprocess.run(f"gunzip -c {folder}/{genome}.fa.gz > {folder}/{genome}.fa", shell=True, check=True)


    tss_bed_file = f"{folder}/{genome}.tss.bed"
    if not os.path.exists(tss_bed_file):
        cmd = f"gunzip -c {folder}/{genome}.refGene.gtf.gz | awk -v OFS='\\t' '{{if($3==\"transcript\"){{if($7==\"+\"){{print $1,$4,$4+1}}else{{print $1,$5-1,$5}}}}}}' | grep -v 'alt' | grep -v 'random' | sort |uniq |sort -k 1,1 -k2,2n > {folder}/{genome}.tss.bed"
    
        print(f"Running {cmd}")
        subprocess.run(cmd, shell=True, check=True)

    binned_bed_file = f"{folder}/{genome}.binned.bed"
    if not os.path.exists(binned_bed_file):
        cmd = f"bedtools makewindows -g {folder}/{genome}.chrom.sizes -w {int(resolution)} > {folder}/{genome}.binned.bed"
        print(f"Running {cmd}")
        subprocess.run(cmd, shell=True, check=True)

    gcpt_bedgraph_file = f"{folder}/{genome}.GCpt.bedGraph"
    if not os.path.exists(gcpt_bedgraph_file):
        cmd = f"bedtools nuc -fi {folder}/{genome}.fa -bed  {folder}/{genome}.binned.bed | grep -v '#' | awk -v OFS='\\t' '{{print $1,$2,$3,$5}}' | grep -v 'alt' | grep -v 'random' | sort -k 1,1 -k2,2n > {folder}/{genome}.GCpt.bedGraph"
        print(f"Running {cmd}")
        subprocess.run(cmd, shell=True, check=True)

    gcpt_tss_bedgraph_file = f"{folder}/{genome}.GCpt.tss.bedGraph"
    if not os.path.exists(gcpt_tss_bedgraph_file):
        cmd = f"bedtools map -a {folder}/{genome}.GCpt.bedGraph -b {folder}/{genome}.tss.bed -c 1 -o count -null 0 > {folder}/{genome}.GCpt.tss.bedGraph"
        print(f"Running {cmd}")
        subprocess.run(cmd, shell=True, check=True)
    print("GCpt.tss.bedGraph already exists")
    goldenpath = f"{folder}/{genome}.GCpt.tss.bedGraph"
    return goldenpath

In [4]:
genome_name = "mm9"
resolution_value = 250000
pc_k = 2
data_folder = None
#下载参考基因组文件并生成 GCpt.tss.bedGraph文件（用于后续挑选pc）
goldenpath = download_and_extract(genome_name, data_folder, resolution_value)

download mm9.refGene.gtf.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12.3M  100 12.3M    0     0  2440k      0  0:00:05  0:00:05 --:--:-- 3110k


Running gunzip -c mm9_250000_goldenpathData/mm9.refGene.gtf.gz | awk -v OFS='\t' '{if($3=="transcript"){if($7=="+"){print $1,$4,$4+1}else{print $1,$5-1,$5}}}' | grep -v 'alt' | grep -v 'random' | sort |uniq |sort -k 1,1 -k2,2n > mm9_250000_goldenpathData/mm9.tss.bed
Running bedtools makewindows -g mm9_250000_goldenpathData/mm9.chrom.sizes -w 250000 > mm9_250000_goldenpathData/mm9.binned.bed
Running bedtools nuc -fi mm9_250000_goldenpathData/mm9.fa -bed  mm9_250000_goldenpathData/mm9.binned.bed | grep -v '#' | awk -v OFS='\t' '{print $1,$2,$3,$5}' | grep -v 'alt' | grep -v 'random' | sort -k 1,1 -k2,2n > mm9_250000_goldenpathData/mm9.GCpt.bedGraph
Running bedtools map -a mm9_250000_goldenpathData/mm9.GCpt.bedGraph -b mm9_250000_goldenpathData/mm9.tss.bed -c 1 -o count -null 0 > mm9_250000_goldenpathData/mm9.GCpt.tss.bedGraph
GCpt.tss.bedGraph already exists


In [5]:
def pc_flip_and_selection(df,use_rows):

    gcc = pd.read_table(goldenpath, header=None, names=["chr", "start", "end", "gcc", "tss"])
    # 为GCC数据添加行名
    gcc.index = gcc["chr"].astype(str) + "_" + gcc["start"].astype(str)
    lhs = pd.DataFrame()
    lhs[['pc1','pc2']] = df
    lhs.index= ['chr1'+'_'+str(start*250000) for start in use_rows]
    lhs[["gcc", "tss"]] = gcc.loc[lhs.index, ["gcc", "tss"]]
    lhs['len'] = range(1, lhs.shape[0] + 1)
    # print(lhs)
    pc_sign = []
    pc_k = 2
    for k in range(pc_k):
            # 计算pc值和gcc的相关性 决定pc值是否乘以-1  
        a = np.sign(pearsonr(lhs.iloc[:, k], lhs["gcc"])[0])
        lhs.iloc[:, k] = a*lhs.iloc[:, k]
        pc_sign.append(a)
    pc_flip= lhs.iloc[:,0:pc_k]

    gcc_values=[pd.concat([pc_flip, lhs["gcc"]],axis=1).corr().iloc[:-1, -1].round(4).transpose()]
    tss_values=[pd.concat([pc_flip, lhs["tss"]],axis=1).corr().iloc[:-1, -1].round(4).transpose()]
    len_values=[pd.concat([pc_flip, lhs["len"]],axis=1).corr().iloc[:-1, -1].round(4).transpose()]
    score = [pc1+pc2 for pc1,pc2 in zip(gcc_values,tss_values)]
    max_pc_index = score.index(max(score))
    pc_flip = np.array(pc_flip.iloc[:,max_pc_index])
    # pc_flip = pc_flip.iloc[:,max_pc_index]
    pc_sign = pc_sign[max_pc_index]

    return pc_flip,max_pc_index,pc_sign

In [6]:
def process_one_chrom(chrom):

	pc_select =None
	pc_flap = None

	# Get the raw sparse mtx list
	cell_type_info=pickle.load(open(os.path.join(temp_dir, "label_info.pickle"), "rb"))
	cell_type=cell_type_info['cell_type']
	origin_sparse = np.load(os.path.join(temp_dir, "%s_sparse_adj.npy" % chrom), allow_pickle=True)
	size = origin_sparse[0].shape[0]
	print(size)
	# find centromere & gaps...
	mask, split_point = create_mask((int(1e5)), chrom, origin_sparse)

	bulk1 = np.array(np.sum(origin_sparse, axis=0).todense())
	print(bulk1.shape)
	mask = (np.ones_like(bulk1) - mask)
	# bulk1 *= mask
	# bulk1 *= mask
	# numpy.core._exceptions._UFuncOutputCastingError: Cannot cast ufunc 'multiply' output from dtype('float64') to dtype('int32') with casting rule 'same_kind'
	bulk1 *= mask.astype(np.int32)
	bulk2 =None


	use_rows_all = []

	if split_point >= 20 * 1000000 / res:
		slice_start_list, slice_end_list = [0, split_point], [split_point, len(bulk1)]
	else:
		slice_start_list, slice_end_list = [0], [len(bulk1)]

	bulk_compartment_all = []
	real_bulk_compartment_all = []
	temp_compartment_list_zscore = []
	temp_compartment_list_quantile = []

	bulk_model_list = []
	bulk_reverse_list = []
	bulk_slice_list = []
	use_rows_list = []
	
	# temp_compartment_list_zscore = []
	# temp_compartment_list_quantile = []

	for slice_start, slice_end in zip(slice_start_list, slice_end_list):
		
		bulk1_slice = bulk1[slice_start:slice_end, :]
		bulk1_slice = bulk1_slice[:, slice_start:slice_end]
		use_rows = np.where(np.sum(bulk1_slice > 0, axis=-1) > 0.01 * len(bulk1_slice))[0]
		if len(use_rows) <= 1:
			print("no reliable bins in slice:", slice_start, slice_end)
			continue
		use_rows_all.append(np.arange(slice_start, slice_end)[use_rows])
		use_rows_list.append(use_rows)
		bulk1_slice = bulk1_slice[use_rows, :]
		bulk1_slice = bulk1_slice[:, use_rows]
		# print('use_rows_list',len(use_rows_list))
		
		bulk_slice_list.append(bulk1_slice)
		bulk_expect = []
		for k in range(len(bulk1_slice)):
			diag = np.diag(bulk1_slice, k)
			bulk_expect.append(np.mean(diag))
		
		
		bulk_compartment, model = test_compartment(bulk1_slice, return_PCA=True)
			
		# reverse_flag = False
		bulk_compartment_all.append(bulk_compartment)
		bulk_model_list.append(model)
	bulk_compartment = np.concatenate(bulk_compartment_all, axis=0)
	use_rows = np.concatenate(use_rows_all, axis=0)
	#bulk_compartment内容为Pc1 pc2
	real_bulk_compartment,max_pc_index,pc_sign= pc_flip_and_selection(bulk_compartment,use_rows)
	print(bulk_compartment.shape)
	print(len(use_rows_list))
	#开始计算每个cell的pc
	temp_compartment_list_all = [[] for i in range(len(use_rows_list))]
	cell_list = trange(len(origin_sparse))
	# cell_list = trange(3)
	# print(cell_list)
	temp = np.zeros((size, size))
	for i in cell_list:
		temp *= 0.0
		proba = np.array(origin_sparse[i].todense())
		temp+= proba
		temp = temp + temp.T
		temp *= mask
		for j in range(len(use_rows_list)):
			slice_start, slice_end = slice_start_list[j], slice_end_list[j]
			temp_slice = temp[slice_start:slice_end, :]
			temp_slice = temp_slice[:, slice_start:slice_end]
			temp_select = temp_slice[use_rows_list[j], :]
			temp_select = temp_select[:, use_rows_list[j]]
			# temp_select = rankmatch(temp_select, bulk_slice_list[j])
			temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
			temp_compartment = temp_compartment[:,max_pc_index]*pc_sign
			# print(type(temp_compartment))
			# print(len(use_rows_list[j]))
			temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
			# temp_compartment_list_all[j].append(temp_compartment)
				
	for j in range(len(use_rows_list)):
		temp_compartment_list_all[j] = np.stack(temp_compartment_list_all[j], axis=0)
		temp_compartment_list_quantile.append(quantile_transform(temp_compartment_list_all[j], output_distribution='uniform',
		                                           n_quantiles=int(temp_compartment_list_all[j].shape[-1] * 1.0), axis=1))
		
		temp_compartment_list_zscore.append(zscore(temp_compartment_list_all[j], axis=1))
	temp_compartment_list = np.concatenate(temp_compartment_list_all, axis=-1)

	temp_compartment_list_zscore = np.concatenate(temp_compartment_list_zscore, axis=-1)
	temp_compartment_list_quantile = np.concatenate(temp_compartment_list_quantile, axis=-1)
	#按细胞类型计算伪bulk的pc值 用于输入dchic
	origin_bulk_list_all=[[] for i in range(len(use_rows_list))]
	# sum_by_label=[]
	a={}
	b={}
	for j in list(set(cell_type)):
		
		indices = [index for index, value in enumerate(cell_type) if value == j]
		a[j]=indices
		b[j] = np.zeros_like(np.array(origin_sparse[0]))
		for i in a[j]:
				proba = np.array(origin_sparse[i])    
				b[j] +=proba
		temp = np.array(b[j].item().todense())
		# print(temp.shape)
		temp *= mask
		for j in range(len(use_rows_list)):
			slice_start, slice_end = slice_start_list[j], slice_end_list[j]
			temp_slice = temp[slice_start:slice_end, :]
			temp_slice = temp_slice[:, slice_start:slice_end]
			temp_select = temp_slice[use_rows_list[j], :]
			temp_select = temp_select[:, use_rows_list[j]]
			# temp_select = rankmatch(temp_select, bulk_slice_list[j])
			temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
			temp_compartment = temp_compartment[:,max_pc_index]*pc_sign #pc值校正
			
			# temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
			origin_bulk_list_all[j].append(temp_compartment.reshape((-1)))
        
	for j in range(len(use_rows_list)):
		origin_bulk_list_all[j] = np.stack(origin_bulk_list_all[j], axis=0)
	presudo_bulk_list = np.concatenate(origin_bulk_list_all, axis=-1)
	print (chrom, "finished")
	
	return presudo_bulk_list, temp_compartment_list,temp_compartment_list_zscore,temp_compartment_list_quantile,use_rows,size


'/home/dataset/snm3c/250k/raw'

In [7]:
def start_call_compartment(output):

	
	if ".hdf5" not in output:
		output += ".hdf5"
	with h5py.File(os.path.join(temp_dir, output), "w") as output_f:
		result = {}
		for chrom in chrom_list:
			presudo_bulk_list, temp_compartment_list,temp_compartment_list_zscore,temp_compartment_list_quantile,use_rows,size = process_one_chrom(chrom)
			result[chrom] = [presudo_bulk_list, temp_compartment_list,temp_compartment_list_zscore,temp_compartment_list_quantile,use_rows,size]
			
		bin_chrom_list = []
		bin_start_list = []
		bin_end_list = []
		presudo_bulk_all = []
		sc_cp_raw = []
		sc_cp_zscore = []
		sc_cp_quantile = []
		#####################################整个数据集的bulk值##########################################################
		# grp = output_f.create_group('compartment')
		# bin = grp.create_group('bin')
		#############################################################################################################
		
		for chrom in chrom_list:
			presudo_bulk_list, temp_compartment_list,temp_compartment_list_zscore,temp_compartment_list_quantile,use_rows,size= result[chrom]
			# print (use_rows)
			# print(chrom)
			length = size
			bin_chrom_list += [chrom] * len(use_rows)
			bin_start_list.append((np.arange(length) * res).astype('int')[use_rows])
			bin_end_list.append(((np.arange(length) + 1) * res).astype('int')[use_rows])
			presudo_bulk_all.append(presudo_bulk_list)
			sc_cp_raw.append(temp_compartment_list)
			sc_cp_zscore.append(temp_compartment_list_zscore)
			sc_cp_quantile.append(temp_compartment_list_quantile)
			
		presudo_bulk_all = np.concatenate(presudo_bulk_all, axis=-1)
		print('presudo_bulk_all.shape:',presudo_bulk_all.shape)
		sc_cp_raw = np.concatenate(sc_cp_raw, axis=-1)
		sc_cp_zscore = np.concatenate(sc_cp_zscore, axis=-1)
		sc_cp_quantile=np.concatenate(sc_cp_quantile, axis=-1)
		
		grp = output_f.create_group('compartment_raw')
		bin = grp.create_group('bin')
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		for cell in range(len(sc_cp_raw)):
			grp.create_dataset("cell_%d" % cell, data=sc_cp_raw[cell])

		grp = output_f.create_group('compartment_zscore')
		bin = grp.create_group('bin')
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		for cell in range(len(sc_cp_zscore)):
			grp.create_dataset("cell_%d" % cell, data=sc_cp_zscore[cell])

		grp = output_f.create_group('compartment')
		bin = grp.create_group('bin')
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		for cell in range(len(sc_cp_quantile)):
			grp.create_dataset("cell_%d" % cell, data=sc_cp_quantile[cell])



		grp = output_f.create_group('compartment_presudo_bulk')
		bin = grp.create_group('bin')
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		cell_type_info=pickle.load(open(os.path.join(temp_dir, "label_info.pickle"), "rb"))
		cell_type=cell_type_info['cell_type']
		cell_info = list(set(cell_type))
		for j in range(len(cell_info)):
			grp.create_dataset(cell_info[j], data=presudo_bulk_all[j,:])
			
	output_f.close()

In [11]:
temp_dir = '/home/dataset/cellcycle/sparse_input/250k/raw'
cytoband_path = '/home/annotation/mm9/cytoBand.txt'
res = 250000
output = 'compartment250k'
chrom_list = ["chr1","chr2","chr3","chr4","chr5",
			  "chr6","chr7","chr8","chr9","chr10",
			  "chr11","chr12","chr13","chr14","chr15",
			  "chr16","chr17","chr18","chr19","chrX"]
start_call_compartment(output)


789
(789, 789)
(769, 2)
1


100%|██████████| 1171/1171 [03:36<00:00,  5.42it/s]


chr1 finished
727
(727, 727)
(710, 2)
1


100%|██████████| 1171/1171 [03:14<00:00,  6.01it/s]


chr2 finished
639
(639, 639)
(627, 2)
1


...

chr18 finished
246
(246, 246)
(234, 2)
1


100%|██████████| 1171/1171 [00:23<00:00, 49.37it/s]


chr19 finished
667
(667, 667)
(636, 2)
1


100%|██████████| 1171/1171 [02:54<00:00,  6.71it/s]


chrX finished
presudo_bulk_all.shape: (4, 10216)


In [127]:
presudo_bulk_list,real_bulk_compartment,temp_compartment_list,temp_compartment_list_zscore,temp_compartment_list_quantile,use_rows = process_one_chrom('chr1')

998
(998, 998)
                     pc1       pc2       gcc  tss  len
chr1_0          2.775440  2.079056  0.324984    8    1
chr1_250000     2.018534  3.482645  0.287844    2    2
chr1_500000     2.030299  1.674580  0.392268    5    3
chr1_750000    -3.298628  2.530375  0.567700   17    4
chr1_1000000   -5.836934  3.942275  0.612900   22    5
...                  ...       ...       ...  ...  ...
chr1_248000000 -5.273661  3.806374  0.387672   12  909
chr1_248250000 -5.327095  3.586650  0.372096    8  910
chr1_248500000 -4.522876  3.032983  0.378568   11  911
chr1_248750000 -4.222943  3.003602  0.250820    7  912
chr1_249000000  3.321932  0.622575  0.328424    7  913

[913 rows x 5 columns]
(913, 2)
2


100%|██████████| 4238/4238 [17:38<00:00,  4.00it/s]


chr1 finished


In [129]:
temp_compartment_list_zscore.shape

(4238, 913)

In [128]:
presudo_bulk_list.shape

(14, 913)

In [104]:
temp_compartment_list.shape

(2, 913)

In [106]:
len(use_rows)

913

In [87]:
max_pc_index

0

In [21]:
        pca[["gcc", "tss"]] = gcc.loc[pca.index, ["gcc", "tss"]]
        pca['len'] = range(1, pca.shape[0] + 1)
        # print(pca.iloc[:, 4])
        # 生成bedGraph文件
        for k in range(4, pca.shape[1] - 3):

            # 计算pc值和gcc的相关性 决定pc值是否乘以-1
            pca.iloc[:, k] = np.sign(pearsonr(pca.iloc[:, k], pca["gcc"])[0])*pca.iloc[:, k]

        # 存储染色体的pc值(正负校正后)
        chrom_list[j] = pca.iloc[:, 4:(pca.shape[1] - 3)]


NameError: name 'bulk_compartment' is not defined

In [37]:
lhs.iloc[:,1]

chr1_0            2.079056
chr1_250000       3.482645
chr1_500000       1.674580
chr1_750000       2.530375
chr1_1000000      3.942275
                    ...   
chr1_227000000    3.806374
chr1_227250000    3.586650
chr1_227500000    3.032983
chr1_227750000    3.003602
chr1_228000000    0.622575
Name: pc2, Length: 913, dtype: float64

In [71]:
lhs = pd.DataFrame()
lhs[['pc1','pc2']] = bulk_compartment
lhs.index= ['chr1'+'_'+str(start*250000) for start in range(bulk_compartment.shape[0])]
lhs[["gcc", "tss"]] = gcc.loc[lhs.index, ["gcc", "tss"]]
lhs['len'] = range(1, lhs.shape[0] + 1)
lhs

,pc1,pc2,gcc,tss,len
chr1_0,2.775440,2.079056,0.324984,8,1
chr1_250000,2.018534,3.482645,0.287844,2,2
chr1_500000,2.030299,1.674580,0.392268,5,3
chr1_750000,-3.298628,2.530375,0.567700,17,4
chr1_1000000,-5.836934,3.942275,0.612900,22,5
...,...,...,...,...,...
chr1_227000000,-5.273661,3.806374,0.446612,3,909
chr1_227250000,-5.327095,3.586650,0.365612,0,910
chr1_227500000,-4.522876,3.032983,0.435116,1,911
chr1_227750000,-4.222943,3.003602,0.445060,8,912


In [74]:
pc_sign = []

In [75]:
pc_k = 2
for k in range(pc_k):
         # 计算pc值和gcc的相关性 决定pc值是否乘以-1  
     a = np.sign(pearsonr(lhs.iloc[:, k], lhs["gcc"])[0])
     lhs.iloc[:, k] = a*lhs.iloc[:, k]
     pc_sign.append(a)

In [77]:
pc_sign[0]

-1.0

In [73]:
a = np.sign(pearsonr(lhs.iloc[:, 0], lhs["gcc"])[0])
# b=a*2
print(a)

-1.0


In [43]:
lhs

,pc1,pc2,gcc,tss,len
chr1_0,-2.775440,-2.079056,0.324984,8,1
chr1_250000,-2.018534,-3.482645,0.287844,2,2
chr1_500000,-2.030299,-1.674580,0.392268,5,3
chr1_750000,3.298628,-2.530375,0.567700,17,4
chr1_1000000,5.836934,-3.942275,0.612900,22,5
...,...,...,...,...,...
chr1_227000000,5.273661,-3.806374,0.446612,3,909
chr1_227250000,5.327095,-3.586650,0.365612,0,910
chr1_227500000,4.522876,-3.032983,0.435116,1,911
chr1_227750000,4.222943,-3.003602,0.445060,8,912


In [44]:
pc_flip= lhs.iloc[:,0:pc_k]
pc_flip

,pc1,pc2
chr1_0,-2.775440,-2.079056
chr1_250000,-2.018534,-3.482645
chr1_500000,-2.030299,-1.674580
chr1_750000,3.298628,-2.530375
chr1_1000000,5.836934,-3.942275
...,...,...
chr1_227000000,5.273661,-3.806374
chr1_227250000,5.327095,-3.586650
chr1_227500000,4.522876,-3.032983
chr1_227750000,4.222943,-3.003602


In [52]:
pc_flip= lhs.iloc[:,0:pc_k]

gcc_values=[pd.concat([pc_flip, lhs["gcc"]],axis=1).corr().iloc[:-1, -1].round(4).transpose()]
        # print(gcc_values)
tss_values=[pd.concat([pc_flip, lhs["tss"]],axis=1).corr().iloc[:-1, -1].round(4).transpose()]
len_values=[pd.concat([pc_flip, lhs["len"]],axis=1).corr().iloc[:-1, -1].round(4).transpose()]

score = [pc1+pc2 for pc1,pc2 in zip(gcc_values,tss_values)]

max_pc_index = score.index(max(score))

pc_flip = np.array(pc_flip.iloc[:,max_pc_index])



In [56]:
print(gcc_values)
tss_values

[pc1    0.3532
pc2    0.0510
Name: gcc, dtype: float64]


[pc1    0.3437
 pc2   -0.0353
 Name: tss, dtype: float64]

In [ ]:
vals_chrom['gcc.cor']+vals_chrom['tss.cor']

In [62]:

score = [pc1+pc2 for pc1,pc2 in zip(gcc_values,tss_values)]
score


[pc1    0.6969
 pc2    0.0157
 dtype: float64]

In [64]:
max_pc_index = score.index(max(score))
max_pc_index


0

In [69]:
len(np.array(pc_flip.iloc[:,0]))

913

<function list.index(value, start=0, stop=9223372036854775807, /)>

In [19]:
gcc = pd.read_table(goldenpath, header=None, names=["chr", "start", "end", "gcc", "tss"])
# 为GCC数据添加行名
gcc.index = gcc["chr"].astype(str) + "_" + gcc["start"].astype(str)

In [20]:
gcc

,chr,start,end,gcc,tss
chr1_0,chr1,0,250000,0.324984,8
chr1_250000,chr1,250000,500000,0.287844,2
chr1_500000,chr1,500000,750000,0.392268,5
chr1_750000,chr1,750000,1000000,0.567700,17
chr1_1000000,chr1,1000000,1250000,0.612900,22
...,...,...,...,...,...
chrY_58250000,chrY,58250000,58500000,0.000000,0
chrY_58500000,chrY,58500000,58750000,0.000000,0
chrY_58750000,chrY,58750000,59000000,0.210224,0
chrY_59000000,chrY,59000000,59250000,0.382952,2


In [27]:
bulk_compartment = process_one_chrom('chr1')

998
(998, 998)
(486, 2)
(427, 2)
(913, 2)


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
'''
@File    :   PC_selection_and _flipping.py
@Time    :   2024/01/05 16:05:47
@Author  :   cp 
@Version :   1.0
@Desc    :   pc挑选和翻转
'''

import os
import subprocess
import h5py 
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from itertools import chain
import re


import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr


# 激活自动转换为pandas数据框
pandas2ri.activate()

# 安装并导入必要的R包
base = importr('base')
stats = importr('stats')





def download_file_with_curl(url, destination):
    subprocess.run(['curl', '-O', url, '-L'], check=True, cwd=destination)


def download_and_extract(genome, folder, resolution):
    if folder is None or folder.strip() == "":
        folder = f"{genome}_{int(resolution)}_goldenpathData"
        if not os.path.exists(folder):
            os.makedirs(folder)
            current_path = os.getcwd()
            os.chdir(current_path)
        folder = os.path.normpath(folder)
    else:
        folder = os.path.normpath(folder)

    genome_fa_file = f"{folder}/{genome}.fa"
    if not os.path.exists(f"{folder}/cytoBand.txt.gz"):
        print("download cytoBand.txt.gz")
        download_file_with_curl(f"http://hgdownload.cse.ucsc.edu/goldenPath/{genome}/database/cytoBand.txt.gz", folder)
        

    if not os.path.exists(f"{folder}/{genome}.chrom.sizes"):
        print(f"download {genome}.chrom.sizes")
        download_file_with_curl(f"http://hgdownload.cse.ucsc.edu/goldenPath/{genome}/bigZips/{genome}.chrom.sizes", folder)
    if not os.path.exists(f"{folder}/{genome}.refGene.gtf.gz"):
        print(f"download {genome}.refGene.gtf.gz")
        download_file_with_curl(f"http://hgdownload.cse.ucsc.edu/goldenPath/{genome}/bigZips/genes/{genome}.refGene.gtf.gz", folder)
        

    if not os.path.exists(f"{folder}/{genome}.fa.gz"):
        genome_fa_url = f"http://hgdownload.cse.ucsc.edu/goldenPath/{genome}/bigZips/{genome}.fa.gz"
        print(f"download {genome}.fa.gz")
        download_file_with_curl(genome_fa_url, folder)
    if not os.path.exists(f"{folder}/{genome}.fa"):  
        print(f"Unzipping {genome}.fa")
        #解压文件输出定向到{folder}文件夹下 {genome}.fa文件
        subprocess.run(f"gunzip -c {folder}/{genome}.fa.gz > {folder}/{genome}.fa", shell=True, check=True)


    tss_bed_file = f"{folder}/{genome}.tss.bed"
    if not os.path.exists(tss_bed_file):
        cmd = f"gunzip -c {folder}/{genome}.refGene.gtf.gz | awk -v OFS='\\t' '{{if($3==\"transcript\"){{if($7==\"+\"){{print $1,$4,$4+1}}else{{print $1,$5-1,$5}}}}}}' | grep -v 'alt' | grep -v 'random' | sort |uniq |sort -k 1,1 -k2,2n > {folder}/{genome}.tss.bed"
    
        print(f"Running {cmd}")
        subprocess.run(cmd, shell=True, check=True)

    binned_bed_file = f"{folder}/{genome}.binned.bed"
    if not os.path.exists(binned_bed_file):
        cmd = f"bedtools makewindows -g {folder}/{genome}.chrom.sizes -w {int(resolution)} > {folder}/{genome}.binned.bed"
        print(f"Running {cmd}")
        subprocess.run(cmd, shell=True, check=True)

    gcpt_bedgraph_file = f"{folder}/{genome}.GCpt.bedGraph"
    if not os.path.exists(gcpt_bedgraph_file):
        cmd = f"bedtools nuc -fi {folder}/{genome}.fa -bed  {folder}/{genome}.binned.bed | grep -v '#' | awk -v OFS='\\t' '{{print $1,$2,$3,$5}}' | grep -v 'alt' | grep -v 'random' | sort -k 1,1 -k2,2n > {folder}/{genome}.GCpt.bedGraph"
        print(f"Running {cmd}")
        subprocess.run(cmd, shell=True, check=True)

    gcpt_tss_bedgraph_file = f"{folder}/{genome}.GCpt.tss.bedGraph"
    if not os.path.exists(gcpt_tss_bedgraph_file):
        cmd = f"bedtools map -a {folder}/{genome}.GCpt.bedGraph -b {folder}/{genome}.tss.bed -c 1 -o count -null 0 > {folder}/{genome}.GCpt.tss.bedGraph"
        print(f"Running {cmd}")
        subprocess.run(cmd, shell=True, check=True)
    print("GCpt.tss.bedGraph already exists")
    goldenpath = f"{folder}/{genome}.GCpt.tss.bedGraph"
    return goldenpath

def process_and_transpose(gcc_values):
    # 将字典中的每个值转换为 DataFrame 
    dfs = {key: pd.DataFrame(value) for key, value in gcc_values.items()}

    # 使用 pd.concat 在轴 1 上合并 DataFrame
    result_df = pd.concat(dfs.values(), axis=1)

    # 转置 DataFrame
    result_df = result_df.transpose()
    # 在列名后添加 .cor 后缀
    result_df.columns = result_df.columns + '.cor'

    # 添加 'name' 列，该列包含原始 DataFrame 的索引
    result_df['name'] = result_df.index

    return result_df
vals_path = 'Python_vals.txt'

if os.path.exists(vals_path):
    os.remove(vals_path)
def run_r_function(i,pc_mat, sam, gcc_values, tss_values, len_values, chr_list):
    # 将Python对象转换为R对象
    r_pc_mat=pandas2ri.py2rpy(pc_mat)
    # r_pc_mat = robjects.r['as.matrix'](r_pc_mat)
    sam_length = len(sam)
    gcc_values_r = pandas2ri.py2rpy(gcc_values)
    tss_values_r = pandas2ri.py2rpy(tss_values)
    len_values_r = pandas2ri.py2rpy(len_values)

    # 在R环境中定义r_pc_mat
    robjects.globalenv['i'] = i
    robjects.globalenv['r_pc_mat'] = r_pc_mat
    robjects.globalenv['sam_length'] = sam_length
    # 在设置 chr_list 到 R 环境之前，将其转换为字符向量
    robjects.globalenv['chr_list'] = robjects.StrVector(chr_list)

    # robjects.globalenv['chr_list'] = chr_list
    robjects.globalenv['gcc_values_r'] = gcc_values_r
    robjects.globalenv['tss_values_r'] = tss_values_r
    robjects.globalenv['len_values_r'] = len_values_r
    

    # 执行R代码
    robjects.r('''
        # print(head(r_pc_mat))
        # print(chr_list)
        # print(gcc_values_r)
    
        hcl <- hclust(as.dist(round(1-cor(r_pc_mat),4)))
		cl<- list()
		k <- 1
		h <- 0.05   
		while (h < 1) {
			g <- data.frame(group=cutree(hcl, h=h))	
			g[,"name"]  <- rownames(g)
			g[,"name2"] <- paste0(g$group,"-",do.call(rbind,strsplit(g$name,"[.]"))[,1])
			v <- data.frame(member=table(g$name2))
			v$member.Var1 <- do.call(rbind,strsplit(as.character(v$member.Var1),"-"))[,1]
			v <- data.frame(member=table(v$member.Var1))
            # print(g)
			s <- v[v$member.Freq == sam_length,]$member.Var1
			if (length(s) > 0) {

				for(u in 1:length(s)) {
					if (length(unique(g[g$group==s[u],]$name)) == sam_length) {
						cl[[k]] <- g[g$group==s[u],]
						k <- k + 1
					}
				}
			}
			h <- h + 0.025
            # print(paste("Current h value:", h))
            # print(paste("Number of groups:", length(unique(g$group))))
            # print(paste("Number of unique s values:", length(unique(s))))
		}
        if (length(cl) > 0) {
			cl <- as.data.frame(do.call(rbind, cl))
			cl <- unique(cl)
			cl[,"chr"] <- chr_list[i+1]
			cl[,"gcc.cor"] <- 0
			cl[,"tss.cor"] <- 0

			for(v in 1:nrow(gcc_values_r)) {
				cl[cl$name == gcc_values_r$name[v],"gcc.cor"] <- gcc_values_r$gcc.cor[v]
				cl[cl$name == gcc_values_r$name[v],"tss.cor"] <- tss_values_r$tss.cor[v]
				cl[cl$name == gcc_values_r$name[v],"len.cor"] <- abs(len_values_r$len.cor[v])
			}
			data$clus[[d]] <- cl
			data$cor[[d]] <- as.data.frame(aggregate(gcc.cor ~ group + chr, mean, data=cl))
			data$cor[[d]][,"tss.cor"] <- aggregate(tss.cor ~ group + chr, mean, data=cl)$tss.cor
			data$cor[[d]][,"len.cor"] <- aggregate(len.cor ~ group + chr, mean, data=cl)$len.cor
			data$cor[[d]][,"score"] <- apply(data$cor[[d]][,c("tss.cor","gcc.cor")],1,sum)
			data$cor[[d]]$gcc.cor <- round(data$cor[[d]]$gcc.cor, 4)
			data$cor[[d]]$tss.cor <- round(data$cor[[d]]$tss.cor, 4)
			data$cor[[d]]$len.cor <- round(data$cor[[d]]$len.cor, 4)
			data$cor[[d]]$score   <- round(data$cor[[d]]$score, 4)
			data$chrom[[d]] <- chr_list[i+1]
			d <- d + 1
		}
        length_cl<-length(cl)''')

    # 从R中检索结果
    data= robjects.globalenv['data']
    length_cl = robjects.globalenv['length_cl']
    # data_clus = robjects.globalenv['data'].rx2('clus')
    # data_cor = robjects.globalenv['data'].rx2('cor')
    # data_chrom = robjects.globalenv['data'].rx2('chrom')
    # print(data_clus)
    # 将R数据框转换为Pandas数据框
    # data_clus = pandas2ri.ri2py(data_clus)
    # data_cor = pandas2ri.ri2py(data_cor)
    # data_chrom = pandas2ri.ri2py(data_chrom)
    # 使用提取数据的方法
    # data_clus_df = pandas2ri.rpy2py_dataframe(data_clus)
    # data_cor_df = pandas2ri.rpy2py_dataframe(data_cor)
    # data_chrom_df = pandas2ri.rpy2py_dataframe(data_chrom)

    return data,length_cl
robjects.r('''
        d <- 1
        data <- list(clus=list(),cor=list(),chrom=list())
        ''')


genome_name = "mm10"
resolution_value = 250000
pc_k = 2
data_folder = None
#下载参考基因组文件并生成 GCpt.tss.bedGraph文件（用于后续挑选pc）
goldenpath = download_and_extract(genome_name, data_folder, resolution_value)
group_id = 'compartment_raw'
df = '/home/python/higashi/dataset_hic/dataset2/cortex250k/compartment_raw.h5'
file = h5py.File(df, 'r')
bin_chr = np.array([chrom.decode('utf-8') for chrom in file['compartment_raw']['bin']['chrom']])
bin_start = file[group_id + '/bin/start'][:]
bin_end = file[group_id + '/bin/end'][:]

chr_list = list(np.unique(bin_chr))
chr_list.sort(key=lambda l: int(re.findall('\d+', l)[0]))
# chr_list = ['chr1','chr2']
print(chr_list)
vals = {}

cell_id = [1,2,3]
sam = ['cell_' + str(cell) for cell in cell_id]

for i in range(len(chr_list)):
    chr_index = np.where(bin_chr == chr_list[i])[0]
    gcc_values = {}
    tss_values = {}
    len_values = {}
    chrom_list = {}
    count_vect = {}

    for j in range(len(sam)):
        print(f"在 {sam[j]} 样本中运行 {chr_list[i]}")
        cell_data = file[group_id + f'/cell_{j}/'][:]
        #按染色体名取对应位置数据
        pca = pd.DataFrame({'chr': bin_chr[chr_index], 'start': bin_start[chr_index], 'end': bin_end[chr_index],'index': bin_end[chr_index]/resolution_value})
        lhs = pd.DataFrame(data=cell_data)
        # print(pca.shape)
        pca[['pc1', 'pc2', 'pc3']] = lhs.iloc[chr_index, :].values
        pca =pca.iloc[:,0:(4+pc_k)]
        
        #添加行名如：chr_250000
        pca.index = pca["chr"].astype(str) + "_" + pca["start"].astype(str)
        
        gcc = pd.read_table(goldenpath, header=None, names=["chr", "start", "end", "gcc", "tss"])
        # 为GCC数据添加行名
        gcc.index = gcc["chr"].astype(str) + "_" + gcc["start"].astype(str)

        # print(pca)

        # 将GCC和TSS信息添加到PCA中
        pca[["gcc", "tss"]] = gcc.loc[pca.index, ["gcc", "tss"]]
        pca['len'] = range(1, pca.shape[0] + 1)
        # print(pca.iloc[:, 4])
        # 生成bedGraph文件
        for k in range(4, pca.shape[1] - 3):

            # 计算pc值和gcc的相关性 决定pc值是否乘以-1
            pca.iloc[:, k] = np.sign(pearsonr(pca.iloc[:, k], pca["gcc"])[0])*pca.iloc[:, k]

        # 存储染色体的pc值(正负校正后)
        chrom_list[j] = pca.iloc[:, 4:(pca.shape[1] - 3)]
        
        colnames_chrom = [f"{sam[j]}.PC{i}" for i in range(1, chrom_list[j].shape[1] + 1)]
        # print(colnames_chrom)
        chrom_list[j].columns = colnames_chrom

        # 存储行名
        count_vect[j] = chrom_list[j].index
        # print(count_vect[j])
        # 计算pc值和GCC、TSS、行数的相关性
        gcc_values[j]=[pd.concat([chrom_list[j], pca["gcc"]], axis=1).corr().iloc[:-1, -1].round(4).transpose()]
        # print(gcc_values)
        tss_values[j]=[pd.concat([chrom_list[j], pca["tss"]], axis=1).corr().iloc[:-1, -1].round(4).transpose()]
        len_values[j]=[pd.concat([chrom_list[j], pca["len"]], axis=1).corr().iloc[:-1, -1].round(4).transpose()]
        # print(gcc_values[j])

    gcc_df=process_and_transpose(gcc_values)
    tss_df = process_and_transpose(tss_values)
    len_df = process_and_transpose(len_values)
    #合并相关性结果
    vals[i] = pd.DataFrame()
    vals[i]['gcc.cor'] = gcc_df['gcc.cor']
    vals[i]['tss.cor'] = tss_df['tss.cor']
    vals[i]['len.cor'] = len_df['len.cor']
    vals[i]['chr'] = [chr_list[i]]*len(gcc_df['gcc.cor'])
    vals[i]['sample'] = vals[i].index.str.split('.').str[0]
    vals[i]['pc']= vals[i].index.str.split('.').str[1]

    vals[i].to_csv(vals_path, sep="\t", header = False,index=False,mode='a')
    # print(vals[i])

    # print(count_vect[0]+count_vect[1])
    #字典中的列表转换成pd.Series并合并成一列
    # values, counts = np.unique( pd.concat([pd.Series(count_vect[0]), pd.Series(count_vect[1])], ignore_index=True), return_counts=True)
    #改成适用于一个或多个列表
    values, counts = np.unique(pd.concat([pd.Series(lst) for lst in count_vect.values()], ignore_index=True), return_counts=True)
    # 将结果转换为数据框
    count_vect_df = pd.DataFrame({'value': values, 'freq': counts})
    # print(count_vect_df)
    # print(count_vect_df)
    pc_mat=[]
    # 遍历 sam 列表的每个元素
  
    for j in range(len(sam)):
        # 选择 chrom_list 中的第 j 个元素的子集
        subset = chrom_list[j].loc[count_vect_df['value'], :]

        # 将子集添加到 pc_mat 列表中
        pc_mat.append(subset)
    # 使用 numpy.c_ 将 pc_mat 列表中的矩阵按列合并
    pc_mat_combined = np.c_[tuple(pc_mat)]

    # 将结果转换为 DataFrame 行索引使用count_vect_df['value']列 列名使用vals[i].index
    pc_mat_df = pd.DataFrame(pc_mat_combined,index=count_vect_df['value'],columns=vals[i].index)


    data,length_cl = run_r_function(i,pc_mat_df, sam, gcc_df, tss_df, len_df, chr_list)

robjects.r('''
    clus.df <- as.data.frame(unique(do.call(rbind,data$clus)))
	cor.df  <- as.data.frame(do.call(rbind,data$cor))
	chr.vec <- unlist(data$chrom)
        ''')
clus_df = pandas2ri.rpy2py_dataframe(robjects.globalenv['clus.df'])
cor_df = pandas2ri.rpy2py_dataframe(robjects.globalenv['cor.df'])

#############################处理完所有染色体后
chr_max = []
for i in range(len(chr_list)):
    cor_df_chrom = cor_df[cor_df['chr']==chr_list[i]]
    clus_df_chrom = clus_df[clus_df['chr']==chr_list[i]]
    if len(cor_df_chrom)>0:
        selected_rows = cor_df_chrom.loc[cor_df_chrom['score'].idxmax()]
        name = clus_df_chrom[clus_df_chrom['group']==selected_rows['group']]['name'].str.split('[.]', expand=True)

    #    name_split = selected_rows['name'].str.split('[.]', expand=True)


        chr_max.append(pd.DataFrame({
            'group': selected_rows['group'],
            'chr': selected_rows['chr'],
            'gcc.cor': selected_rows['gcc.cor'],
            'tss.cor': selected_rows['tss.cor'],
            'len.cor': selected_rows['len.cor'],
            'sample': [','.join(name[0])],
            'pcs': [','.join(map(str, name[1]))]
        }))
    # print(chr_list[i])
    else:
        vals_chrom = vals[i]
        vals_chrom['score'] = vals_chrom['gcc.cor']+vals_chrom['tss.cor']
        # print(vals_chrom)
        sample_df = {}
        #选择每个sample中score最大的pc所在的行
        for j in range(len(sam)):
            sample_df[j] = vals_chrom[vals_chrom['sample']==sam[j]]
            
            sample_df[j] = sample_df[j].loc[sample_df[j]['score'].idxmax()]
            
            # print(type(sample_df[j]))
        #按行合并并转换成数据帧
        sample_df = pd.DataFrame(pd.concat(sample_df,axis=1).T)
        # print(sample_df)
        # 计算每列的均值
        gcc_cor_mean = round(sample_df['gcc.cor'].mean(), 4)
        tss_cor_mean = round(sample_df['tss.cor'].mean(), 4)
        len_cor_mean = round(sample_df['len.cor'].mean(), 4)
        
        # 将结果添加到 chr_max 列表中
        chr_max.append(pd.DataFrame({
            'group': [1],
            'chr': [chr_list[i]],
            'gcc.cor': [gcc_cor_mean],
            'tss.cor': [tss_cor_mean],
            'len.cor': [len_cor_mean],
            'sample': [','.join(sample_df['sample'])],
            'pcs': [','.join(map(str, sample_df['pc']))]
        }))
# 使用 pd.concat 将列表中的数据框按行合并
chr_max_df = pd.concat(chr_max, ignore_index=True)

# 将数据框写入文本文件
chr_max_df.to_csv("Python_test_chr_pc_selected1.txt", sep="\t", index=False)